In [1]:
# -*- coding: utf-8 -*-
"""
Preparação de Dados para Dashboard no Power BI
--------------------------------------------------

Objetivo:
 1. Processar e estruturar os dados do conjunto train_df para facilitar a análise no Power BI.
 2. Criar DataFrames separados que podem ser utilizados como tabelas de suporte no Power BI.
 3. Explorar segmentações relevantes baseadas na análise exploratória, incluindo clusters, categorias e estatísticas.
 4. Garantir que os dados estejam organizados para visualizações eficientes e insights estratégicos.
"""

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Carregamento dos dados
train_path = "data/train.csv"
train_df = pd.read_csv(train_path)

# 1. Criação de uma Tabela de Fatos (Fato_Vendas)
fato_vendas = train_df[['Id', 'Neighborhood', 'OverallQual', 'GrLivArea', 'GarageCars',
                         'TotalBsmtSF', 'YearBuilt', 'SalePrice']].copy()

# Adicionar colunas derivadas relevantes para análise
fato_vendas['DecadeBuilt'] = (fato_vendas['YearBuilt'] // 10) * 10  # Década de construção
fato_vendas['PricePerSqFt'] = fato_vendas['SalePrice'] / fato_vendas['GrLivArea']  # Preço por metro quadrado

# 2. Criação de uma Tabela Dimensão para Bairro (Dim_Neighborhood)
dim_neighborhood = train_df.groupby("Neighborhood").agg({
    "SalePrice": "mean",
    "GrLivArea": "mean",
    "LotArea": "mean",
    "TotalBsmtSF": "mean"
}).reset_index()

dim_neighborhood.columns = ["Neighborhood", "AvgSalePrice", "AvgGrLivArea", "AvgLotArea", "AvgTotalBsmtSF"]

# Clusterização de bairros baseado no preço médio
dim_neighborhood_scaled = StandardScaler().fit_transform(dim_neighborhood[['AvgSalePrice', 'AvgGrLivArea']])
kmeans = KMeans(n_clusters=4, random_state=42)
dim_neighborhood['Cluster'] = kmeans.fit_predict(dim_neighborhood_scaled)

# 3. Criação de uma Tabela Dimensão para Qualidade Geral (Dim_Qualidade)
dim_qualidade = train_df.groupby("OverallQual").agg({
    "SalePrice": "mean",
    "GrLivArea": "mean",
    "TotalBsmtSF": "mean"
}).reset_index()

dim_qualidade.columns = ["OverallQual", "AvgSalePrice", "AvgGrLivArea", "AvgTotalBsmtSF"]

# 4. Criação de uma Tabela Dimensão para Ano de Construção (Dim_YearBuilt)
dim_yearbuilt = train_df.groupby("YearBuilt").agg({
    "SalePrice": "mean",
    "GrLivArea": "mean"
}).reset_index()

dim_yearbuilt.columns = ["YearBuilt", "AvgSalePrice", "AvgGrLivArea"]

# Exportação das tabelas para CSV para uso no Power BI
fato_vendas.to_csv("data/fato_vendas.csv", index=False)
dim_neighborhood.to_csv("data/dim_neighborhood.csv", index=False)
dim_qualidade.to_csv("data/dim_qualidade.csv", index=False)
dim_yearbuilt.to_csv("data/dim_yearbuilt.csv", index=False)

# Exibir visualização de estrutura
print("\n===== Estrutura dos DataFrames Criados =====")
print("Fato_Vendas:", fato_vendas.head())
print("\nDim_Neighborhood:", dim_neighborhood.head())
print("\nDim_Qualidade:", dim_qualidade.head())
print("\nDim_YearBuilt:", dim_yearbuilt.head())

"""
Resumo das Tabelas Criadas:
- **Fato_Vendas:** Contém as transações de venda, incluindo preços e métricas relevantes.
- **Dim_Neighborhood:** Estatísticas agregadas por bairro, incluindo clusterização.
- **Dim_Qualidade:** Estatísticas agregadas baseadas na qualidade geral dos imóveis.
- **Dim_YearBuilt:** Estatísticas agregadas baseadas no ano de construção.

Essas tabelas são ideais para serem usadas em dashboards do Power BI, permitindo análises de preço por região,
qualidade da construção e tendências históricas.
"""



===== Estrutura dos DataFrames Criados =====
Fato_Vendas:    Id Neighborhood  OverallQual  GrLivArea  GarageCars  TotalBsmtSF  \
0   1      CollgCr            7       1710           2          856   
1   2      Veenker            6       1262           2         1262   
2   3      CollgCr            7       1786           2          920   
3   4      Crawfor            7       1717           3          756   
4   5      NoRidge            8       2198           3         1145   

   YearBuilt  SalePrice  DecadeBuilt  PricePerSqFt  
0       2003     208500         2000    121.929825  
1       1976     181500         1970    143.819334  
2       2001     223500         2000    125.139978  
3       1915     140000         1910     81.537566  
4       2000     250000         2000    113.739763  

Dim_Neighborhood:   Neighborhood   AvgSalePrice  AvgGrLivArea    AvgLotArea  AvgTotalBsmtSF  \
0      Blmngtn  194870.882353   1427.941176   3398.176471     1324.411765   
1      Blueste  137500.

'\nResumo das Tabelas Criadas:\n- **Fato_Vendas:** Contém as transações de venda, incluindo preços e métricas relevantes.\n- **Dim_Neighborhood:** Estatísticas agregadas por bairro, incluindo clusterização.\n- **Dim_Qualidade:** Estatísticas agregadas baseadas na qualidade geral dos imóveis.\n- **Dim_YearBuilt:** Estatísticas agregadas baseadas no ano de construção.\n\nEssas tabelas são ideais para serem usadas em dashboards do Power BI, permitindo análises de preço por região,\nqualidade da construção e tendências históricas.\n'